### Word Frequency

In [284]:
import string
import pandas as pd
import numpy as np
import nltk 
nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)
from functools import reduce

[nltk_data] Downloading package stopwords to /Users/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [258]:
postsDF = pd.read_csv('./scrappy/scrappy_do.csv')

def wordCount(post):
    d = {}
    l = str(post).split(" ")
    filtered = [w for w in l if w not in stop_words]
    for w in filtered:
        w = w.translate(translator) # remove punctuation
        w = w.lower()
        if (d.get(w)):
            d[w] = d[w] + 1
        else:
            d[w] = 1
    return d

oneString = ' '.join(list(map(lambda x: str(x), list(postsDF['message']))))
wordCounts = wordCount(oneString)

In [264]:
# ordered words by frequency count to determine a list of important words
wordCountDF = pd.DataFrame.from_dict(wordCounts, orient="index")
print(list(wordCountDF.sort_values(by=0, ascending=False).reset_index()['index']))

['i', '', 'car', 'the', 'like', 'cars', 'bmw', 'would', 'one', 'it', 'get', 'new', 'think', 'tl', 'even', 'much', 'im', 'better', 'good', 'drive', 'well', '3', 'really', 'know', 'and', 'people', 'time', 'also', 'price', 'audi', 'want', 'performance', 'but', 'if', 'years', 'still', 'acura', 'driving', 'you', 'see', 'could', 'that', 'way', 'is', 'luxury', 'make', 'back', 'say', 'go', 'lexus', 'engine', 'series', 'thats', 'buy', 'going', 'sedan', 'g35', 'not', 'many', 'so', 'great', 'sport', 'less', 'ive', 'my', 'manual', 'miles', 'right', 'best', 'said', 'year', 'me', 'got', 'in', 'awd', 'take', 'lot', 'interior', 'its', 'look', 'infiniti', 'never', 'cts', 'may', 'sure', 'us', 'since', 'cant', 'nice', 'they', 'dealer', 'test', 'thing', 'first', 'need', 'something', 'probably', 'point', 'two', 'power', 'this', 'little', 'last', 'every', 'looking', 'rwd', 'around', 'id', 'used', 'now', 'feel', 'tires', '2', 'money', 'a', 'least', 'class', 'actually', '5', 'there', 'cost', 'old', 'seems', '

In [281]:
modelsDF = pd.read_csv('./models.csv', names=['make', 'model'])

model_to_make = modelsDF.set_index('model').to_dict(orient='dict')['make']

importantWords = ['performance', 'luxury sport', 'quality','experience' ,'power','handling',
                  'premium',  'reliable', 'expensive', 'worth', 'speed', 'family']
brands = list(model_to_make.values())
keepWords = list(set(importantWords + brands))

def stopUnique(post):
    # lowercase
    post = str(post).lower()
    
    # remove punctuation
    post = post.translate(translator)
    modelKeys = model_to_make.keys()
    wordList = post.split(' ')
    
    # remove stop words
    filtered = [w for w in wordList if w not in stop_words]
    
    # only keep important words
    finalWords = list(filter(lambda x: x in keepWords, filtered))
    
    # make / model replacement
    for idx, word in enumerate(finalWords):
        if word in modelKeys:
            finalWords[idx] = model_to_make.get(word)
            
    unique = list(set(finalWords))            
    return unique if len(unique) else np.nan

postsDF['words'] = postsDF['message'].head().map(lambda x: stopUnique(x))
postsDF.dropna(inplace=True)
postsDF.head()

,date,userid,message,words
0,2002-03-25T05:54:02+00:00,merc1,I personally think that with a few tweaks the ...,"[acura, ford, problem, power, toyota, car, bui..."
1,2002-03-25T07:06:29+00:00,fredvh,I am debating a new purchase and these two are...,"[acura, worth, honda, quality]"
2,2002-03-25T17:02:27+00:00,blueguydotcom,"Great handling, RWD, excellent engine and the ...",[handling]
4,2002-03-26T00:44:13+00:00,riez,One beauty of BMW 3 Series is that there are s...,"[performance, sedan, bmw]"


In [285]:
dataset = list(postsDF['words'])

In [286]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.0, use_colnames=True)


frequent_itemsets


,support,itemsets
0,0.50,(acura)
1,0.25,(audi)
2,0.50,(bmw)
3,0.25,(buick)
4,0.25,(car)
5,0.25,(chrysler)
6,0.25,(ford)
7,0.50,(handling)
8,0.25,(honda)
9,0.25,(mercedes-benz)


In [ ]:
from mlxtend.frequent_patterns import association_rules
result_lift = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)


/Users/matt/anaconda3/lib/python3.7/site-packages/mlxtend/frequent_patterns/association_rules.py:105: RuntimeWarning: invalid value encountered in double_scalars
  "confidence": lambda sAC, sA, _: sAC/sA,


In [34]:
result_lift.sort_values(by=['lift'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
155,"(think, car)",(dont),0.092411,0.188654,0.042467,0.459540,2.435891,0.025033,1.501215
158,(dont),"(think, car)",0.188654,0.092411,0.042467,0.225104,2.435891,0.025033,1.171239
157,(think),"(car, dont)",0.176791,0.102336,0.042467,0.240209,2.347264,0.024375,1.181462
156,"(car, dont)",(think),0.102336,0.176791,0.042467,0.414975,2.347264,0.024375,1.407135
102,(know),(dont),0.124123,0.188654,0.054699,0.440684,2.335939,0.031283,1.450605
103,(dont),(know),0.188654,0.124123,0.054699,0.289944,2.335939,0.031283,1.233532
107,(dont),(want),0.188654,0.101320,0.041682,0.220944,2.180656,0.022568,1.153550
106,(want),(dont),0.101320,0.188654,0.041682,0.411390,2.180656,0.022568,1.378409
132,"(car, dont)",(cars),0.102336,0.220504,0.048144,0.470456,2.133546,0.025579,1.472012
133,(cars),"(car, dont)",0.220504,0.102336,0.048144,0.218338,2.133546,0.025579,1.148405
